# Step 5: Add a deployment pipeline
In previous four steps you implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

You can use a [SageMaker MLOps project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html) to provision a ready-to use model deployment CI/CD pipeline.

This template automates the deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. This template recognizes changes in the model registry. When a new model version is registered and approved, it automatically initiates a deployment.

![](img/six-steps-5.png)

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

#sagemaker.__version__

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                        -> 's3://customer-churning-sage-maker/pipeline_notebo
domain_id                              -> 'd-o9e9qxxwhi5b'
evaluation_s3_url                      -> 's3://customer-churning-sage-maker/pipeline_notebo
experiment_name                        -> 'from-idea-to-prod-experiment-15-11-26-05'
input_s3_url                           -> 's3://customer-churning-sage-maker/dataset/input/b
mlflow_arn                             -> 'arn:aws:sagemaker:ap-south-1:356652681186:mlflow-
mlflow_name                            -> 'sage-maker-project'
model_package_group_name               -> 'from-idea-to-prod-pipeline-model-22-09-26-41'
output_s3_url                          -> 's3://customer-churning-sage-maker/pipeline_notebo
pipeline_name                          -> 'from-idea-to-prod-pipeline-22-09-26-41'
prediction_baseline_s3_url             -> 's3://customer-churning-sage-maker/pipeline_notebo
space_name            

In [3]:
sm = boto3.client("sagemaker")

## Create an MLOps project
Follow the same procedure as in the step 4 notebook to create a model deployment MLOps project. 

Option 1 is recommended as it requires no manual input and has no dependency on the UX.</br>
Option 2 is given to demonstrate [**Create Project** UI flow](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html).

### Option 1: Create project programmatically
Use `boto3` to create an MLOps project via a SageMaker API.

In [4]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model deployment"

In [5]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [6]:
p_ids

['prod-5vmc7unyqksbs']

In [7]:
# If you get any exception from this code, go to the Option 2 and create a project in Studio UI
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

ML Ops product id: prod-5vmc7unyqksbs


In [8]:
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [9]:
provisioning_artifact_id

'pa-q4qe35zerlzhq'

In [10]:
project_name = f"model-deploy-{strftime('%-m-%d-%H-%M-%S', gmtime())}"

In [11]:
project_parameters = [
    {
        'Key': 'SourceModelPackageGroupName',
        'Value': model_package_group_name
    },
]

Finally, create a SageMaker project from the service catalog product template:

In [12]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:398333718380:project/model-deploy-7-23-11-46-25', 'ProjectId': 'p-ilq3pvs0je1i', 'ResponseMetadata': {'RequestId': '82b38ed5-69c7-4272-a606-729a794d4279', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '82b38ed5-69c7-4272-a606-729a794d4279', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Tue, 23 Jul 2024 11:46:25 GMT'}, 'RetryAttempts': 0}}


<div class="alert alert-info"> 💡 <strong> Wait until project creation is completed by running the next cell</strong>
</div>

In [13]:
# Project creation takes about 3-5 min
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    sleep(10)
    print("Waiting for project creation completion")

print(f"MLOps project {project_name} creation completed")

Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
MLOps project model-deploy-7-23-11-46-25 creation completed


### End of Option 1: Create project programmatically
Now you have provisioned a project template in your SageMaker environment. Navigate to the section **Working with MLOps project for model deployment**.

---

### Option 2: Create a project in Studio UI
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

In [15]:
print(f"The last used model package group is {model_package_group_name}")

The last used model package group is from-idea-to-prod-pipeline-model-08-08-30-16


Follow the instructions in the Developer Guide – [Create a MLOps Project using Amazon SageMaker Studio or Studio Classic](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html). Choose the **Studio** option.

For the template choose the **Model deployment**.
In the **Project details** you need to provide a name and an optional project description. 

This template has a required parameter `SourceModelPackageGroupName`. Put the model package group name you used in the step 3 and 4 notebooks.

Optionally, add tags, which are key-value pairs that you can use to track your projects.

Choose **Create** and wait for the project to appear in the Projects list.

### End of Option 2: Create a project in Studio UI
Now when you have the project created, move to the section **Working with MLOps project for model deployment**.

---

## Working with MLOps project for model deployment
The template provisions a CodeCommit repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. An AWS CodeCommit repository that contains template code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the CodeCommit repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected.
4. There is also a manual approval step after the placeholder unit tests. You can implement your own tests to replace the placeholders tests.

The template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

### Approve a model version
Approving a model version causes the MLOps project to launch the model deployment process. 

In the first step, the model deployment pipeline deploys the model version to a staging SageMaker real-time inference end-point.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

In [14]:
try:
    print(model_package_group_name)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Run the step 03 notebook to create a pipeline, run the pipeline, and register a model version in the model registry")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

from-idea-to-prod-pipeline-model-23-07-51-02


In [15]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

There are 5 model versions in the from-idea-to-prod-pipeline-model-23-07-51-02 model package group
Approve the most recent model package:
arn:aws:sagemaker:us-east-1:398333718380:model-package/from-idea-to-prod-pipeline-model-23-07-51-02/5


The following statement sets the `ModelApprovalStatus` for the most recent model package in the model registry to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [16]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

You can see the last model version in the model registry in the Studio UI changed the **Status** to `Approved`:

![](img/model-registry-version-approved.png)

### Deployment pipeline execution
Upon approval of a model version in the code cell above, the model deployment CI/CD pipeline performs the following actions:
1. Create CloudFormation parameter configuration files with stating and prod parameters for CloudFormation templates with SageMaker endpoint IaC
1. Create a SageMaker real-time inference endpoint with the name `<PROJECT-NAME>-staging` in the current account
1. Run the test script on the staging endpoint
1. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

Wait about 10-15 minutes until the pipeline finishes deployment of the staging endpoint. You can see the status of the endpoint in the Studio UI in **Deployments** > **Endpoints**:

![](img/endpoint-staging-creating.png)

After the endpoint status changed from `Creating` to `InService`, the staging endpoint is fully operational. You can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline. In the next section you approve the model deployment and launch the second stage of the deployment into a production endpoint.

To see the inference endpoints in the Studio UI click on the link constructed by the code cell below:

In [19]:
from IPython.display import HTML

# Show the infernece endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

### Deploy the model version to production
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">Wait until staging endpoint status changes to InService, then continue with the following code cells.
    </p>
</div>

Let's construct a CodePipeline approval link. 

If you used the option 1 `boto3` to create an MLOps project, the `project_name` and `project_id` are set automatically. You can run the following code cell to print the values. If you followed the UI instructions to create a project, you must set the `project_name` manually.

In [20]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++")
    print("You must set the project_name manually")
    print("++++++++++++++++++++++++++++++++++++++")

model-deploy-7-23-11-46-25
p-ilq3pvs0je1i


In [21]:
# Set to the model deployment project name if you didn't use boto3-based deployment
# project_name = "<USE YOUR PROJECT NAME>"

# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [22]:
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

Click on the link ^^^ above ^^^ to open the CodePipeline console with the pipeline execution workflow.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. Note, you might wait until `TestStaging` step completes with `Succeeded` status. 

![](img/deploy-staging-review.png)

In the **Review** dialog box, select **Approve** and choose **Submit**:

![](img/approve-deployment.png)

Approving the **DeployStaging** stage causes the deployment pipeline to continue and to deploy the model to the production endpoint. To view the endpoints, choose the **Deployments** > **Endpoints** in Studio UI.

In [23]:
# Show the infernece endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

As your CI/CD deployment pipeline continues, you see the production endpoint in status `Creating` along with the previously deployed staging endpoint in status `InService`:

![](img/endpoint-prod-creating.png)

After 10-15 min the deployment is completed and both endpoints are in status `InService`.

Navigate to the Studio and choose **Deployments** > **Projects**. In the Project pane select `model-deploy-<TIMESTAMP>` project. In the project details pane select **Endpoints**. You see that both endpoints, `staging` and `prod`, are visible in the deployment project because the project and the endpoints are connected via the metadata:

![](img/project-endpoints.png)


## Summary
In this notebook you implemented an automated CI/CD deployment pipeline with the following features:
- use CloudFormation IaC templates for SageMaker real-time inference endpoint deployment
- model approval in the model registry launches the model deployment pipeline
- model deployment pipeline contains two stages, staging and production with automated tests for the staging endpoint and manual approval for the production deployment

---

## Clean-up
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    If you're going to run the step 6 notebook (Data and Model Quality Monitoring), you need to keep at least one of the endpoints. If you finish the workshop here and don't run the step 6 notebook, navigate to the <b>clean-up notebook (99-clean-up.ipynb)</b> and follow the clean-up instructions to avoid charges in your AWS account.
    <br>
    <br>
    You don't need to run the clean-up if you're using an AWS-provided AWS account.
    </p>
</div>

## Continue with the step 6
open the step 6 [notebook](06-monitoring.ipynb).

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>